# Second Pass

This script addresses two issues

Data cleansing and filtering is an iterative process and this contains a second tranche of filtering to remove 
1.Rows where location accuracy is zero. It transpires that this means unknown rather than zero! 
2.Remove rows where the location type start is 2 (GeoIP rather than GPS) 
3.Any location_age_a which is older than ten minutes. (10 x 60 x 1000 milliseconds)

Another issue was that it was not possible to load the (half) coverage map without closing down all other applications 
Purpose of this was to create two pickle format files which with all of the required pre-processing. 

Loading the (half) coverage map consumed 1.2GB memory. It wasn’t possible to load this without closing down all other applications but changing the data type from float to integer halves memory consumption. Therefore this script also writes the fully filtered Ookla data and the coverage map to pickle format files so they can be re-loaded into the final modelling script. 





In [1]:
# load libraries
import pandas as pd
import numpy as np

This script carries out final preparation of the coverage map and Ookla data  
                                                          
COVERAGE MAP
============
This section:  
1) Loads the first 12,000 lines of the August coverage map. (This is the max which can be loaded but 
   covers 93.7% of the sample data.)
2) Changes the data from floats to integer. This reduces memory consumption from 1.2GB to 600MB 
3) The lowest actual value is -127 dBm but no signal at all is recorded as 0 (which as loss is being measured would
   represent a perfect signal!) Changed zeros to -128 to prevent them distorting any predictions.  
4) Rewrites the coverage map as a pickle file.  


In [2]:
# set some pointers
workdir = 'p:/ookla/pickle'
rasterfile = 'P:/ookla/4G_RSS/2021-01-08-AW_H3G_LTE1800-1st-Best-RSS.asc'


In [3]:
# define a function to show shape and and memory consumption of dataframe
def sizeof(df):
    name = [x for x in globals() if globals()[x] is df][0]
    print('Dataframe name:', name)
    print('Shape:', df.shape)
    print('Memory consumed: {0:.2f} MB'.format(
        df.memory_usage().sum() / (1024 * 1024)
    ))


In [4]:
# Run task manager, kill your browsers and look at memory consumption before running this!
# file contains 22830 rows (checked with wc -l) and we want to read the last 12k only
rf = pd.read_csv(rasterfile, delim_whitespace=True, header=None, low_memory=False, skiprows=22830 - 12000)

print(sizeof(rf))

Dataframe name: rf
Shape: (12000, 13561)
Memory consumed: 1241.55 MB
None


In [5]:
# Check the mean values before changing floats to integer 
print('mean values in the dataframe are', rf.mean().mean())

mean values in the dataframe are -44.87305983457463


In [6]:
rf = rf.astype(int)
print('mean values in the dataframe are (still)', rf.mean().mean())
print(sizeof(rf))

mean values in the dataframe are (still) -44.87305983457463
Dataframe name: rf
Shape: (12000, 13561)
Memory consumed: 620.77 MB
None


In [7]:
# Check the mean values again after changing floats to integer 
print('mean values in the dataframe are', rf.mean().mean())

mean values in the dataframe are -44.87305983457463


In [8]:
# Make a copy of the map as a pickle file (memory efficient)
rf = rf.replace(0, -128)
rf.to_pickle(workdir + '/aug_map.pickle')

Final Ookla data preparation
============================
Data prep is always an iterative process. This second tranche of filtering will takes as its input 
the concatenated monthly files from May, Jun, Jul, Aug and remove: 
1)  Rows further north than 600,000 (600km). This removes 2,914 of 46810 records = 6.225% (actually this happens in the initial filtering stage!)
2)  Rows where location accuracy is zero. It transpires that this means unknown rather than zero! 
3)	Select only rows where location_type_start is 1 (GPS rather than GeoIP)
4)	Remove rows where location_age_a is older than ten minutes. (10 x 60 x 1000 milliseconds)
5)  Rewrite the 24x pre_connection_type codes as either WiFi, Cellular or unknown then use get_dummies to create cats.  
6)  Rewrite the Ookla data as a pickle file.



In [10]:
# read the Ookla CSV file
datafile = 'P:/ookla/may-aug.csv' 
ook_df = pd.read_csv(datafile, low_memory=False)
print(ook_df.shape)

(473141, 38)


In [11]:
""" 
1) Rows further north than 600,000 (600km)
THIS FILTER IS DEPLOYED IN FIRST PASS. On test data (single day) this removed 2,914 of 46810 records, about 6.225%.  
Confirm that all the test data is below 600km North
"""
print('Max Northings value is', ook_df['N'].max())


Max Northings value is 599976.7267282322


In [12]:
# Data happens to have a couple of unused columns so rather than delete and create rename these as
# RSRP (signal strength) and WiFi (Boolean categorical for WiFi connection)
ook_df = ook_df.rename(columns={'Unnamed: 0': 'RSRP', 'index': 'WiFi'})
print('starting shape', ook_df.shape)

starting shape (473141, 38)


In [13]:
# 2)  Rows where location accuracy is zero. It transpires that this means unknown rather than zero!
# It turns out that location accuracy of "0" doesn't mean what you might think!
# Remove all the rows where this is zero!
print('removing location accuracy = 0 (which means unknown rather than zero!)')
ook_df = ook_df[ook_df['location_accuracy_a'] > 0]
print('Shape is now', ook_df.shape)

removing location accuracy = 0 (which means unknown rather than zero!)
Shape is now (457736, 38)


In [14]:
# 3)	Select only rows where location_type_start is 1 (GPS rather than GeoIP)
# 1=GPS
# 2=GeoIP
print('Selecting only rows which use GPS location')
ook_df = ook_df[ook_df['location_type_start'] == 1]
print('Shape is now', ook_df.shape)

Selecting only rows which use GPS location
Shape is now (457452, 38)


In [15]:
# 4)	Remove rows where location_age_a is older than ten minutes. (10 x 60 x 1000 milliseconds)
print('Removing any location_age_a older than ten minutes.')
# First change 
ook_df.location_age_a = pd.to_numeric(ook_df.location_age_a, errors='coerce').fillna(0).astype(np.int64)
ook_df = ook_df[ook_df['location_age_a'] < 10 * 60 * 1000]
print('Shape is now', ook_df.shape)

Removing any location_age_a older than ten minutes.
Shape is now (456059, 38)


In [16]:
# 5)  Use the pre_connection_type codes to determine which calls are on WiFi
# 2 = WiFi, all else is some type of cellular
ook_df['WiFi'] = np.where(ook_df.pre_connection_type == 2, 1, 0)

In [17]:
# 6)  Rewrite Ookla data as a pickle file.
ook_df.to_pickle(workdir + '/ookla.pickle')

In [18]:
ook_df.to_csv(workdir + '/ookla_final.csv')

In [19]:
rf.to_csv(workdir + '/map_final.csv')